In [1]:
# Draws a bounding box around an object found in image
def drawRectangle(read_image, coordinates):
    # Represent all sides of a box
    image = Image.open(read_image)
    draw = ImageDraw.Draw(image)
    draw.rectangle(coordinates, outline='red')
    image.show()

In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [10]:
from PIL import Image, ImageDraw
read_image = open("../images/image4.png", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(1)


Waiting for result...


In [33]:

from PIL import ImageOps
image = Image.open(read_image)
draw = ImageDraw.Draw(image)

grayImage = ImageOps.grayscale(image)

colorHasher={}
counter=0
lineSum = 0
topbox = 0
bottombox = 0
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            r=(counter%3==0)*5*(counter//3+5)+150
            g=(counter%3==1)*5*(counter//3+5)+150
            b=(counter%3==2)*5*(counter//3+5)+150
            print(line.text)
            lineSum += line.bounding_box[0]
            print(line.bounding_box)
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline=(r,g,b))
            if counter==0:topbox=line.bounding_box[1]
            bottombox=line.bounding_box[5]
            counter+=1

marginLineLeft=10
for top in range(10,int(topbox)):
    for left in range(10,int(lineSum)//counter+10):
        if image.getpixel((left,top))!=(255,255,255):print(image.getpixel((left,top)))
        if sum(image.getpixel((left,top)))<500:
            print(left,top)
            marginLineLeft=left
            break
for top in range(int(bottombox),image.height-10):
    for left in range(10,int(lineSum)//counter+10):
        if sum(image.getpixel((left,top)))<500:
            marginLineLeft=(left+marginLineLeft)//2
            break
draw.line((marginLineLeft,0,marginLineLeft,image.height-1), fill=(255,0,0))

lineAvg = lineSum / counter
print(lineAvg)
image.show()
print()

Register No. 1518/02131
[977.0, 84.0, 1352.0, 74.0, 1353.0, 116.0, 977.0, 131.0]
Course code : UISCS 927 R
[977.0, 164.0, 1360.0, 172.0, 1359.0, 210.0, 977.0, 204.0]
Course name : Cloud Computing
[980.0, 227.0, 1411.0, 228.0, 1410.0, 271.0, 980.0, 268.0]
Date : 11/02 / 2022
[985.0, 306.0, 1343.0, 308.0, 1343.0, 359.0, 984.0, 358.0]
No. of pages : 15
[984.0, 391.0, 1250.0, 398.0, 1248.0, 447.0, 984.0, 440.0]
PART - A
[220.0, 526.0, 377.0, 537.0, 373.0, 589.0, 217.0, 577.0]
Cloud computing :
[223.0, 641.0, 564.0, 649.0, 563.0, 716.0, 222.0, 709.0]
In cloud computing , the client does not have to
[364.0, 752.0, 1399.0, 793.0, 1398.0, 844.0, 362.0, 806.0]
buy The resources and manage thing . Instead , the provider.
[231.0, 876.0, 1395.0, 885.0, 1394.0, 942.0, 231.0, 930.0]
provides the resources as a Service which is much cheaper .
[217.0, 979.0, 1323.0, 1002.0, 1322.0, 1058.0, 216.0, 1029.0]
Essentiel characteristics.
[263.0, 1091.0, 754.0, 1093.0, 754.0, 1146.0, 263.0, 1143.0]
# On deman